In [1]:
import pandas as pd
import numpy as np

In [157]:
# 데이터 Load
# encoding='euc-kr' : 한글 데이터를 불러올 수 있게 한다.
df = pd.read_csv('서울교통공사승하차인원(20200101_20200229).csv', encoding='euc-kr')
#del df['역번호']
df[df['역명'].isin(station_in_bucheon)]

,날짜,호선,역번호,역명,구분,06:00 이전,06:00 ~ 07:00,07:00 ~ 08:00,08:00 ~ 09:00,09:00 ~ 10:00,...,15:00 ~ 16:00,16:00 ~ 17:00,17:00 ~ 18:00,18:00 ~ 19:00,19:00 ~ 20:00,20:00 ~ 21:00,21:00 ~ 22:00,22:00 ~ 23:00,23:00 ~ 24:00,24:00 이후
498,2020-01-01,7호선,2753,까치울,승차,42,74,89,142,182,...,312,263,259,214,161,140,133,68,24,1
499,2020-01-01,7호선,2753,까치울,하차,10,87,70,86,91,...,239,209,295,256,245,219,258,219,182,0
500,2020-01-01,7호선,2754,부천종합운동장,승차,49,41,62,218,156,...,183,163,145,103,104,72,81,47,11,1
501,2020-01-01,7호선,2754,부천종합운동장,하차,24,148,120,58,57,...,127,119,150,127,128,90,113,101,96,1
502,2020-01-01,7호선,2755,춘의,승차,85,101,118,176,207,...,257,206,219,243,174,132,87,86,29,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168389,2020-10-31,7호선,2756,신중동,하차,72,166,199,372,543,...,630,695,874,848,682,526,492,418,351,68
168390,2020-10-31,7호선,2757,부천시청,승차,141,203,344,509,491,...,700,784,691,572,515,517,421,364,139,3
168391,2020-10-31,7호선,2757,부천시청,하차,70,174,202,334,551,...,665,742,740,724,627,422,418,364,307,66
168392,2020-10-31,7호선,2758,상동,승차,104,191,336,448,495,...,804,830,812,783,566,525,563,470,200,4


In [133]:
# 2020-10-01 ~ 2020-11-30 까지의 데이터 사용
start_date = '2020-10-01'
end_date = '2020-11-30'
df_d = df[(df['날짜'] >= start_date) & (df['날짜'] <= end_date)]

In [134]:
# 부천의 데이터 사용
station_in_bucheon = ('까치울','부천종합운동장','춘의','신중동','부천시청','상동')
df_db = df_d[ df_d.역명.isin(station_in_bucheon) ]

In [142]:
# 날짜별, 역별로 승하차 데이터 합치기
df_dbd = df_db.groupby(by=['날짜', '역명'], as_index=False).sum()

# 날짜별로 승하차 데이터 합치기
#df_dbd = df_db.groupby(by=['날짜'], as_index=False).sum()

In [147]:
# 모든 시간대의 승하차 데이터 합치기
df_dbd['승하차총원'] = df_dbd.apply(lambda x: sum(x[2:22]), axis=1)
df_dbds = df_dbd.iloc[:,[0,1,-1]]

In [148]:
df_dbds

,날짜,역명,승하차총원
0,2020-10-01,까치울,4148
1,2020-10-01,부천시청,5748
2,2020-10-01,부천종합운동장,2577
3,2020-10-01,상동,5855
4,2020-10-01,신중동,6176
...,...,...,...
181,2020-10-31,부천시청,18931
182,2020-10-31,부천종합운동장,5969
183,2020-10-31,상동,21390
184,2020-10-31,신중동,19206


In [149]:
def subway_passenger_date(csv_path, start_date='2020-11-01', end_date='2020-11-30', per_station=False) :
    # 파일 읽기 및 불필요 column삭제
    df = pd.read_csv(csv_path, encoding='euc-kr')
    del df['역번호']

    # 조회 기간에 해당하는 행 조회
    df_d = df[(df['날짜'] >= start_date) & (df['날짜'] <= end_date)]
    
    # 부천의 역들의 통계만 보도록 한다.
    station_in_bucheon = ('까치울','부천종합운동장','춘의','신중동','부천시청','상동')
    df_db = df_d[ df_d.역명.isin(station_in_bucheon) ]
    
    if per_station:
        # 날짜별, 역별로 승하차 데이터 합치기
        df_dbd = df_db.groupby(by=['날짜', '역명'], as_index=False).sum()
        df_dbd['승하차총원'] = df_dbd.apply(lambda x : sum(x[2:22]), axis=1)
        df_dbds = df_dbd.iloc[:,[0,1,-1]]
    else:
        # 날짜별로 승하차 데이터 합치기
        df_dbd = df_db.groupby(by=['날짜'], as_index=False).sum()
        df_dbd['승하차총원'] = df_dbd.apply(lambda x: sum(x[1:21]), axis=1)
        df_dbds = df_dbd.iloc[:,[0,-1]]
    
    return df_dbds

In [154]:
subway_passenger_date('서울교통공사승하차인원(20200101_20200229).csv',
                        start_date='2020-10-01',
                        end_date='2020-10-31')

,날짜,승하차총원
0,2020-10-01,28912
1,2020-10-02,43476
2,2020-10-03,52262
3,2020-10-04,46623
4,2020-10-05,115478
5,2020-10-06,118100
6,2020-10-07,118336
7,2020-10-08,121150
8,2020-10-09,70064
9,2020-10-10,73257


In [ ]:
def subway_passenger_date(csv_path, start_date='2020-11-01', end_date='2020-11-30', per_station=False) :
    # 파일 읽기 및 불필요 column삭제
    df = pd.read_csv(csv_path, encoding='euc-kr')
    del df['역번호']

    # 조회 기간에 해당하는 행 조회
    df_d = df[(df['날짜'] >= start_date) & (df['날짜'] <= end_date)]
    
    # 부천의 역들의 통계만 보도록 한다.
    station_in_bucheon = ('까치울','부천종합운동장','춘의','신중동','부천시청','상동')
    df_db = df_d[ df_d.역명.isin(station_in_bucheon) ]
    
    # 날짜별로 승하차 데이터 합치기
    df_dbd = df_db.groupby(by=['날짜'], as_index=False).sum()
    df_dbd['승하차총원'] = df_dbd.apply(lambda x: sum(x[1:21]), axis=1)
    df_dbds = df_dbd.iloc[:,[0,-1]]
    
    return df_dbds